In [1]:
from azureml.core import Workspace
ws = Workspace.from_config()

## Select Target Compute

In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.exceptions import ComputeTargetException

cluster_name = 'mutaz-gpu'
cluster_name = 'gpucluster'

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    compute_target.wait_for_completion(show_output=True)
except ComputeTargetException as e:
    print(e)

Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## Execution Env and Data Access

* Use a GPU enabled curated env specifications
* Present dataset as a mount point, pass it through command line arguements

In [3]:
from azureml.core import ScriptRunConfig
from azureml.core import Environment
from azureml.core import Dataset


# load dataset object and preset as a mount point to 
input_dataset = Dataset.get_by_name(ws, name='cats_dogs')

env = Environment.get(workspace=ws, name='AzureML-tensorflow-2.4-ubuntu18.04-py37-cuda11-gpu')
#  name='AzureML-tensorflow-2.7-ubuntu20.04-py38-cuda11-gpu') #, version=9)
train_runconfig = ScriptRunConfig(
    source_directory="./src", 
    script='train.py', 
    arguments=['--input_dataset', input_dataset.as_mount()], 
    compute_target=compute_target,
    environment=env)

In [4]:
from azureml.core import Experiment
#print([exp for exp in ws.experiments])
exp = Experiment(ws, 'cats-and-dogs')
run = exp.submit(train_runconfig)

In [5]:
print(run.display_name)

calm_double_2k0d9f92


In [6]:
#run.wait_for_completion(show_output=True)